In [3]:
# SVM Classification
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score

#https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/

In [4]:
from pandas import read_csv

In [5]:
forestfires = read_csv('forestfires.csv')
forestfires

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [10]:
forestfires.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')

In [7]:
#Delete columns 'month', 'day' as it already exists
forestfires.drop(['month', 'day'], axis=1, inplace=True)
forestfires

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,large
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,large
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,large
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small


In [6]:
forestfires['size_category'].unique()

array(['small', 'large'], dtype=object)

In [11]:
#Standardize these columns 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind','rain', 'area',
forestfires.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')

In [16]:
forestfires_stndrd = forestfires[['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']]
forestfires_stndrd


,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
#Making whole data into one format
sc= StandardScaler()
sc.fit(forestfires_stndrd)
forestfires_stndrd = sc.transform(forestfires_stndrd)
forestfires_stndrd

array([[-0.80595947, -1.32332557, -1.83047676, ...,  1.49861442,
        -0.07326831, -0.20201979],
       [-0.00810203, -1.17954077,  0.48889092, ..., -1.74175564,
        -0.07326831, -0.20201979],
       [-0.00810203, -1.04982188,  0.56071545, ..., -1.51828184,
        -0.07326831, -0.20201979],
       ...,
       [-1.64008316, -0.84664771,  0.47476811, ...,  1.49861442,
        -0.07326831, -0.02653216],
       [ 0.68095666,  0.54900254,  0.26938221, ..., -0.00983371,
        -0.07326831, -0.20201979],
       [-2.02087875, -1.68591332, -1.78044169, ...,  0.26950853,
        -0.07326831, -0.20201979]])

In [38]:
forestfires_stndrd[512]

array([-1.64008316, -0.84664771,  0.47476811, -1.56346017,  1.53608362,
       -0.75379965, -0.73612355, -0.07326831, -0.10075273])

In [35]:
forestfires_stndrd[:,8:9]

array([[-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.02019788e-01],
       [-2.0

In [31]:
forestfires['FFMC'] = forestfires_stndrd[:,:1]

In [33]:
forestfires.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')

In [39]:
forestfires['DMC'] = forestfires_stndrd[:,1:2]
forestfires['DC'] = forestfires_stndrd[:,2:3]
forestfires['ISI'] = forestfires_stndrd[:,3:4]
forestfires['temp'] = forestfires_stndrd[:,4:5]
forestfires['RH'] = forestfires_stndrd[:,5:6]
forestfires['wind'] = forestfires_stndrd[:,6:7]
forestfires['rain'] = forestfires_stndrd[:,7:8]
forestfires['area'] = forestfires_stndrd[:,8:9]
forestfires

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,-0.805959,-1.323326,-1.830477,-0.860946,-1.842640,0.411724,1.498614,-0.073268,-0.202020,1,...,0,0,0,0,1,0,0,0,0,small
1,-0.008102,-1.179541,0.488891,-0.509688,-0.153278,-0.692456,-1.741756,-0.073268,-0.202020,0,...,0,0,0,0,0,0,0,1,0,small
2,-0.008102,-1.049822,0.560715,-0.509688,-0.739383,-0.692456,-1.518282,-0.073268,-0.202020,0,...,0,0,0,0,0,0,0,1,0,small
3,0.191362,-1.212361,-1.898266,-0.004756,-1.825402,3.233519,-0.009834,0.603155,-0.202020,1,...,0,0,0,0,1,0,0,0,0,small
4,-0.243833,-0.931043,-1.798600,0.126966,-1.291012,3.356206,-1.238940,-0.073268,-0.202020,0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,-1.640083,-0.846648,0.474768,-1.563460,1.536084,-0.753800,-0.736124,-0.073268,-0.100753,0,...,0,0,0,0,0,0,0,0,0,large
513,-1.640083,-0.846648,0.474768,-1.563460,0.519019,1.638592,0.995798,-0.073268,0.651674,0,...,0,0,0,0,0,0,0,0,0,large
514,-1.640083,-0.846648,0.474768,-1.563460,0.398350,1.577248,1.498614,-0.073268,-0.026532,0,...,0,0,0,0,0,0,0,0,0,large
515,0.680957,0.549003,0.269382,0.500176,1.156839,-0.140366,-0.009834,-0.073268,-0.202020,0,...,0,0,0,0,0,0,0,0,0,small


In [40]:
forestfires_stndrd[0]

array([-0.80595947, -1.32332557, -1.83047676, -0.86094557, -1.84264015,
        0.41172435,  1.49861442, -0.07326831, -0.20201979])

In [42]:
array = forestfires.values
X = array[:,0:28]
Y = array[:,28]
Y

array(['small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'small', 'small', 'small', 'small', 'small', 'small',
       'small', 'sma

In [43]:
X

array([[-0.8059594724698989, -1.3233255688310948, -1.8304767629860488,
        ..., 0, 0, 0],
       [-0.008102033950154875, -1.1795407723785454, 0.48889091543801605,
        ..., 0, 1, 0],
       [-0.008102033950154875, -1.0498218799267889, 0.5607154538443283,
        ..., 0, 1, 0],
       ...,
       [-1.640083158195089, -0.8466477110264475, 0.47476811294239285,
        ..., 0, 0, 0],
       [0.6809566629532638, 0.5490025414966674, 0.2693822137917581, ...,
        0, 0, 0],
       [-2.0208787538522395, -1.685913316407089, -1.7804416912872696,
        ..., 1, 0, 0]], dtype=object)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3)

In [45]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((361, 28), (361,), (156, 28), (156,))

# Grid Search CV

In [46]:
clf = SVC()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5],'C':[15,14,13,12,11,10,0.1,0.001] }]
gsv = GridSearchCV(clf,param_grid,cv=10)
gsv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [15, 14, 13, 12, 11, 10, 0.1, 0.001],
                          'gamma': [50, 5, 10, 0.5], 'kernel': ['rbf']}])

In [47]:
gsv.best_params_ , gsv.best_score_ 

({'C': 11, 'gamma': 0.5, 'kernel': 'rbf'}, 0.7478228228228228)

In [48]:
clf = SVC(C= 11, gamma = 0.5)
clf.fit(X_train , y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred) * 100
print("Accuracy =", acc)
confusion_matrix(y_test, y_pred)

Accuracy = 77.56410256410257


array([[ 20,  22],
       [ 13, 101]], dtype=int64)